In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn import svm



In [15]:
df = pd.read_excel("transformeddata2.xlsx")


In [16]:
df = df.fillna(df.mean())
df1 = df.fillna(method='ffill')


In [17]:
ysplit = df1['Person Type']
xsplit = df1.drop(['Person Type'],axis=1)
dt = xsplit.select_dtypes(include=['datetime64'])
cat2 = list(dt.columns)
categorical = xsplit.select_dtypes(exclude=['float64'])
cat3 = list(categorical.columns)
cols = cat2 + cat3
le = preprocessing.LabelEncoder()
xsplit[cols] = xsplit[cols].apply(lambda col: le.fit_transform(col))


In [18]:
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(xsplit,ysplit)

In [19]:
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(xsplit.columns)
fes = pd.concat([dfcolumns,dfscores],axis=1)
fes.columns = ['Specs','Score']
fes.nlargest(15,'Score')
fes = fes.sort_values(by =['Score'],ascending=False)
large = fes.head(10)

corr = large['Specs'].values
xsplit1 = xsplit[corr]        

In [20]:
xtrain, xtest = train_test_split(xsplit1, test_size=0.2, shuffle=False)
ytrain, ytest = train_test_split(ysplit, test_size=0.2, shuffle=False)

In [21]:
clf = svm.SVC(kernel='rbf')
clf.fit(xtrain, ytrain)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [22]:
ypred = clf.predict(xtest)

In [25]:
result = accuracy_score(ytest,ypred)
print("The accuracy using Support Vector Machines {}".format(result))

The accuracy using Support Vector Machines 0.824


In [24]:
from yellowbrick.classifier import ClassPredictionError, ClassificationReport, ROCAUC, PrecisionRecallCurve, ConfusionMatrix
model = clf
viz = ClassPredictionError(model)
viz.fit(xtrain,ytrain)
viz.score(xtest,ytest)
viz.show()

classes = ["CAR", "SUV", "TRUCK", "VAN"]
viz = ClassificationReport(model, classes=classes, support=True)
viz.fit(xtrain,ytrain)
viz.score(xtest,ytest)
viz.show()
 
classes = ["CAR", "SUV", "TRUCK", "VAN"]
viz = ROCAUC(model, classes=classes)
viz.fit(xtrain,ytrain)
viz.score(xtest,ytest)
viz.show()
 
viz = PrecisionRecallCurve(model)
viz.fit(xtrain,ytrain)
viz.score(xtest,ytest)
viz.show()
 
classes = ["CAR", "SUV", "TRUCK", "VAN"]
viz = ConfusionMatrix(model,classes=classes,label_encoder={0: 'CAR', 1: 'SUV', 2: 'TRUCK', 3: 'VAN'})
viz.fit(xtrain,ytrain)
viz.score(xtest,ytest)
viz.show()

<Figure size 800x550 with 1 Axes>

<Figure size 800x550 with 2 Axes>

<Figure size 800x550 with 1 Axes>

<Figure size 800x550 with 1 Axes>

<Figure size 800x550 with 1 Axes>